# Autoenkoderi

U ovoj svesci biće prikazano kako se autoenkoderi mogu kreirati korišćenjem objektne podrške Keras biblioteke.

In [1]:
import torch
import torch.nn as nn

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras import backend as K

Specifični model dobija se nasleđivanjem `Model` objekta biblioteke Keras. Slojevi koje model objedinjuje se kreiraju u konstruktorskoj metodi `__init__`, a propagacija unapred kroz mrežu se definiše metodom `call`. 

Model koji ćemo kreirati odgovara autoenkoderu konstruisanom u prethodnom primeru. 

In [3]:
class Autoencoder(nn.Module):
    def __init__(self, latent_dim, kernel_size, layer_filters):
        super().__init__()
        
        self.image_size = 28
        
        self.latent_dim = latent_dim
        self.kernel_size = kernel_size
        self.layer_filters = layer_filters
        
        self.encoder = self.__create_encoder(self.image_size)
        
        self.decoder = self.__create_decoder(self.image_size)
        
    def __create_encoder(self, image_size):
        num_channels = 1
        layers = []
        for filters in self.layer_filters:
            layers.append(
                nn.Conv2d(num_channels, filters, kernel_size=self.kernel_size, stride=2, padding='same')
            )
            num_channels = filters
            layers.append(nn.ReLU())
        layers.append(nn.Flatten())
        H, W = image_size

        layers.append(nn.Linear(H * W * num_channels, self.latent_dim))        
        return nn.Sequential(*layers)
        
    def __create_decoder(self, image_size):
        
        H, W = image_size
        layers = [
            nn.Linear(self.latent_dim, H * W * self.layer_filters[-1]),
            nn.ReLU()
        ]
        num_channels = self.layer_filters[-1]
        layers.append(
            nn.Unflatten(1, (num_channels, H, W))
        )
        for filters in reversed(self.layer_filters):
            layers.append(
                nn.ConvTranspose2d(filters, filters, kernel_size=self.kernel_size, stride=2, padding='same')
            )
            layers.append(nn.ReLU())
        layers.append(
            nn.ConvTranspose2d(filters, 1, kernel_size=self.kernel_size, padding='same')
        )
        layers.append(nn.Sigmoid())
        return nn.Sequential(*layers)
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [4]:
latent_dim = 16
kernel_size = 3
layer_filters = [32, 64]

In [5]:
autoencoder = Autoencoder(latent_dim, kernel_size, layer_filters)

Samo kompajliranje modela, treniranje i testiranje mreže se sprovesti na uobičajen način tj. korišćenjem `compile`, `fit` i `predict` funkcija.